# NutriBot: Smart Nutrition Assistant for Personalized Dietary Support

## 1. Install and Import Libraries


In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import fitz
import pandas as pd
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langsmith import Client
from langchain.callbacks.manager import get_openai_callback
from langchain.callbacks.tracers.langchain import LangChainTracer
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from deep_translator import GoogleTranslator
from faster_whisper import WhisperModel
import time
import re
from IPython.display import display
import gradio as gr
import base64


## 2. Load Environment Variables


In [ ]:
# Load environment variables from .env file
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
    print(f"✅ .env loaded from: {dotenv_path}")
else:
    print("❌ .env file not found")

# Enable LangChain advanced tracing and evaluation (LangSmith)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "NutriBot Evaluation"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Set LangSmith API endpoint and API key
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
print("🔑 LangSmith API Key:", LANGCHAIN_API_KEY)

# Load OpenAI API Key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("🔑 OpenAI API Key:", OPENAI_API_KEY)

## 3. Load and Parse Datasets (PDF Books)


In [ ]:
print("📖 Extracting text from all 3 PDF books...")

# Dictionary of book sources and file paths
book_files = {
    "Krause": "Janice L. Raymond, Kelly Morrow - Krause and Mahan’s Food and the Nutrition Care Process, 16E-Elsevier (2022).pdf",
    "PocketGuide": "Essential_Pocket_Guide_Clinical_Nutrition.pdf",
    "Pathophysiology": "Marcia Nelms, Kathryn P. Sucher - Nutrition Therapy and Pathophysiology-Cengage Learning (2019).pdf"
}

# List to hold extracted paragraphs
all_paragraphs = []

# Loop through each book
for source, file in book_files.items():
    try:
        print(f"🔹 Reading: {file}")
        doc = fitz.open(file)  # Open PDF

        book_text = ""
        for page in doc:
            book_text += page.get_text()  # Extract text page by page
        doc.close()

        # Split the raw text into meaningful paragraphs
        paragraphs = [p.strip() for p in book_text.split("\n\n") if len(p.strip()) > 50]

        # Tag each paragraph with its source
        for para in paragraphs:
            all_paragraphs.append({"text": para, "source": source})

        print(f"✅ Extracted {len(paragraphs)} paragraphs from {source}")

    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")

# Convert list to DataFrame
book_df = pd.DataFrame(all_paragraphs)

# Print final stats and sample
print(f"✅ Combined final dataset shape: {book_df.shape}")

if len(book_df) > 0:
    print(book_df.sample(min(5, len(book_df))))
else:
    print("⚠️ Dataset is empty.")


## 4. Build Vector Store (ChromaDB)


In [ ]:
# Convert to LangChain Documents
documents = [
    Document(
        page_content=str(row["text"]),
        metadata={"source": row["source"]}
    )
    for _, row in book_df.iterrows()
]

# Chunk Text into Smaller Passages
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separator="\n"
)

texts = text_splitter.split_documents(documents)
print(f"✅ Split into {len(texts)} text chunks.")

# Generate Embeddings
embedding = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=OPENAI_API_KEY
)

# Store Embeddings in Chroma VectorDB
vectorstore = Chroma.from_documents(
    texts,
    embedding,
    persist_directory="chromaadb/",
    collection_metadata={"hnsw:space": "ip"}
)
vectorstore.persist()
print("✅ Chroma vectorstore created and persisted.")

## 5. Prompt Template & Memory

### Create memory


In [ ]:
# Create memory object to retain conversation history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### Define prompt template

In [ ]:
# Define structured prompt template to prevent hallucination
prompt_template = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template="""
You are a helpful, honest, and well-trained nutrition assistant.

Your task is to answer questions strictly based on the given context. Follow these instructions carefully:

1. **Use ONLY the information provided in the context.**
   - Do not use any external knowledge.
   - Do not assume or guess facts that are not explicitly stated in the context.

2. **If the context does not contain the answer:**
   - Say: "I'm sorry, I don't have enough information to answer that."
   - Do NOT attempt to generate or hallucinate an answer.

3. **Answer clearly and concisely.**
   - Focus on factual, evidence-based information.
   - Avoid speculation or over-explaining.

4. **Maintain a neutral and professional tone.**
   - Avoid overly casual or emotional responses.

---------------------
Chat History:
{chat_history}

Context:
{context}

Question:
{question}

---------------------
Your Answer:"""
)

## 6. Load LLM and Create RetrievalQA Chain

In [ ]:
# Create retriever from vectorstore for similarity-based context fetching
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Load OpenAI GPT-4o model for answering questions
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4o",
    temperature=0.2,
    max_tokens=1024,
    streaming=True
)

In [ ]:
def answer_question(user_question, context):
    if not context or len(context.strip()) < 50:
        return "❌ I'm sorry, I don't have enough information to answer that."

    # Load memory
    memory_variables = memory.load_memory_variables({})
    chat_history_list = memory_variables["chat_history"]

    # Convert chat_history to string
    chat_history_str = "\n".join([
        f"{msg.type.capitalize()}: {msg.content}" for msg in chat_history_list
    ])

    # Format prompt
    final_prompt = prompt_template.format(
        chat_history=chat_history_str,
        context=context,
        question=user_question
    )

    # Invoke LLM manually
    response = llm.invoke(final_prompt)

    # Save this interaction to memory
    memory.chat_memory.add_user_message(user_question)
    memory.chat_memory.add_ai_message(response.content)

    return response.content


## 7. QA Wrapper


In [ ]:
def qa_tool_wrapper(question_text):
    try:
        # 1. استرجاع السياق من ChromaDB
        context_docs = retriever.get_relevant_documents(question_text)

        # 2. فلترة أذكى: نسمح بعبارات عامة إذا كانت تحتوي على كلمات مفتاحية مهمة
        important_keywords = ["blood pressure", "hypertension", "sodium", "salt", "diet", "avoid"]
        filtered_docs = [
            doc for doc in context_docs
            if len(doc.page_content.strip()) > 100 or 
            any(kw in doc.page_content.lower() for kw in important_keywords)
        ]

        # 3. تأكد من وجود محتوى بعد الفلترة
        if not filtered_docs:
            return "❌ I'm sorry, I don't have enough information from trusted sources to answer that."

        # 4. دمج السياق
        context = "\n".join([doc.page_content for doc in filtered_docs])

        # 5. تحميل المحادثة
        chat_history_list = memory.load_memory_variables({})["chat_history"]
        chat_history_str = "\n".join([
            f"{msg.type.capitalize()}: {msg.content}" for msg in chat_history_list
        ])

        # 6. إعداد البرومبت
        final_prompt = prompt_template.format(
            chat_history=chat_history_str,
            context=context,
            question=question_text
        )

        # 7. استدعاء النموذج
        response = llm.invoke(final_prompt)

        # 8. فلترة الكلمات المحظورة ثقافيًا
        banned_terms = [
            r"\b(pork)\b",
            r"\b(bacon)\b",
            r"\b(ham)\b",
            r"\b(prosciutto)\b",
            r"\b(sausage made from pork)\b"
        ]

        # استبدال كل كلمة بجملة فارغة أو ترك الكلمة تنحذف بسلاسة
        for term in banned_terms:
            response.content = re.sub(term, "", response.content, flags=re.IGNORECASE)

        # تنظيف أي فراغ زائد أو فواصل غير منطقية
        response.content = re.sub(r"\s{2,}", " ", response.content)
        response.content = re.sub(r"\s+([.,])", r"\1", response.content)

        # 9. حفظ في الذاكرة
        memory.chat_memory.add_user_message(question_text)
        memory.chat_memory.add_ai_message(response.content)

        return response.content

    except Exception as e:
        return f"System error: {e}"


## 8. Image Captioning with BLIP

In [ ]:
# Use GoogleTranslator to translate text between English and Arabic
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def image_caption(image_path):
    try:
        image = Image.open(str(image_path)).convert('RGB')
        inputs = blip_processor(image, return_tensors="pt")
        out = blip_model.generate(**inputs)
        return blip_processor.decode(out[0], skip_special_tokens=True)
    except Exception as e:
        return f"Image processing error: {e}"

## 9. Translation Between Arabic and English

In [ ]:
# Use GoogleTranslator to translate text between English and Arabic
def translate(text, source_lang="en", target_lang="ar"):
    try:
        return GoogleTranslator(source=source_lang, target=target_lang).translate(text)
    except Exception as e:
        print(f"Translation Error: {e}")
        return text

## 10. Audio Transcription with Whisper

In [ ]:
# Load Whisper models for both English and Arabic transcription
whisper_model_ar = WhisperModel("large", compute_type="auto", device="cpu")
whisper_model_en = WhisperModel("medium", compute_type="auto", device="cpu")

def transcribe_audio(audio_path, language):
    try:
        if language == "ar":
            model = whisper_model_ar
        elif language == "en":
            model = whisper_model_en
        else:
            return "❌ Unsupported language. Use 'ar' or 'en'."

        segments, _ = model.transcribe(audio_path, language=language)
        return " ".join([seg.text for seg in segments])
    except Exception as e:
        print(f"Transcription error: {e}")
        return None

def handle_arabic_question(question_ar):
    question_en = translate(question_ar, "ar", "en")
    answer_en = qa_tool_wrapper(question_en)
    return translate(answer_en, "en", "ar")

## 11. LangChain Tools & Agent Setup


### Define tools

In [ ]:
# Define tools that the agent can call (e.g., QA, image, audio, translation)
tools = [
    Tool(
        name="Disease_Information",
        func=qa_tool_wrapper,
        description="For questions about disease definitions, causes, and risk factors. Uses conversation history."
    ),
    Tool(
        name="Symptom_Checker",
        func=qa_tool_wrapper,
        description="For checking symptoms related to specific conditions. Considers previous discussion."
    ),
    Tool(
        name="Treatment_Advice",
        func=qa_tool_wrapper,
        description="Provides evidence-based treatment recommendations. Maintains context."
    ),
    Tool(
        name="Nutrition_Recommendation",
        func=qa_tool_wrapper,
        description="Suggests personalized nutrition plans based on health conditions and history."
    ),
    Tool(
        name="General_Nutrition_QA",
        func=qa_tool_wrapper,
        description="Answers general nutrition questions while remembering conversation context."
    ),
    Tool(
        name="Image_Analysis",
        func=image_caption,
        description="Analyzes food images and generates descriptions for follow-up questions."
    ),
    Tool(
        name="Audio_Transcription",
        func=transcribe_audio,
        description="Transcribes audio in Arabic or English for use in conversation."
    ),
    Tool(
        name="Translation_Service",
        func=lambda text: translate(text, source_lang="auto", target_lang="en"),
        description="Translates between Arabic and English while preserving context."
    )
]

### Initialize conversational agent

In [ ]:
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    early_stopping_method="force"  # ✅ يمنع GPT من الإجابة إذا لم يعرف أي أداة
)


## 12. LangSmith Evaluation

In [ ]:
# Evaluate responses using LangSmith (latency, cost, hallucination detection)
client = Client()

def evaluate_with_langsmith(user_question_en):
    try:
        start_time = time.time()

        with get_openai_callback() as cb:
            response_dict = agent_executor.invoke({"input": user_question_en})
            end_time = time.time()

        response = response_dict["output"]
        latency = round(end_time - start_time, 2)
        cost = round(cb.total_cost, 4)

        # Simple hallucination/relevance check (customize as needed)
        irrelevant_keywords = [
            "cloud", "server", "AI model", "database", "networking", "API",
            "programming", "code", "machine learning", "software"
        ]
        hallucinated = any(keyword in response.lower() for keyword in irrelevant_keywords)
        relevance = not hallucinated

        # Log evaluation summary
        print(f"📊 Relevance: {relevance}, 👻 Hallucination: {hallucinated}")
        print(f"⏱️ Latency: {latency}s, 💰 Cost: ${cost}")

        return response
    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        return "❌ Sorry, an error occurred during evaluation."

In [ ]:
# Run tests on QA pairs and display metrics in DataFrame
qa_pairs = [
    { "question": "What is IBS?" },
    { "question": "How many calories are in an avocado?" },
    { "question": "Best healthy breakfast for hypertension?" },
    # Hallucination test questions
    { "question": "Is eating glass good for digestion?" },
    { "question": "Does eating magnets align your blood flow?" },
    { "question": "Can plugging your ears help reduce sugar levels?" },
]

# Collect evaluation results
evaluation_results = []

# Evaluate each question
for item in qa_pairs:
    question_text = item["question"]
    print(f"\n🔍 Testing: {question_text}")

    start_time = time.time()
    with get_openai_callback() as cb:
        try:
            response_dict = agent_executor.invoke({"input": question_text})
            end_time = time.time()

            response = response_dict["output"]
            latency = round(end_time - start_time, 2)
            cost = round(cb.total_cost, 4)

            # Simple hallucination check
            hallucinated = any(word in response.lower() for word in [
                "motor oil", "fishtea", "ai model", "cloud", "network", "code"
            ])
            relevance = not hallucinated

            # Store the result
            evaluation_results.append({
                "Question": question_text,
                "Answer": response,
                "Relevance": relevance,
                "Hallucination": hallucinated,
                "Latency (s)": latency,
                "Cost ($)": cost
            })

            print(f"📊 Relevance: {relevance}, 👻 Hallucination: {hallucinated}")
            print(f"⏱️ Latency: {latency}s, 💰 Cost: ${cost}")

        except Exception as e:
            print(f"❌ Error during evaluation: {e}")
            evaluation_results.append({
                "Question": question_text,
                "Answer": "ERROR",
                "Relevance": False,
                "Hallucination": True,
                "Latency (s)": None,
                "Cost ($)": None
            })

# Show the evaluation results in a DataFrame
df_eval = pd.DataFrame(evaluation_results)
display(df_eval)

## 13. Deployment

### Main Chatbot Logic for Handling Text, Voice, and Image Inputs


In [ ]:
def chatbot_logic(message, input_type, uploaded_image, audio_path, health_condition, language_choice):
    user_question = ""
    transcription_display = None

    if input_type == "Text":
        user_question = message

    elif input_type == "Voice":
        if audio_path is None:
            return "❌ No voice provided.", None
        language_code = "ar" if language_choice == "Arabic" else "en"
        try:
            user_question = transcribe_audio(audio_path, language=language_code)
            transcription_display = user_question
            if not user_question or len(user_question.strip()) < 3:
                return "❌ The transcription was empty or unclear. Please try speaking again.", None
        except Exception as e:
            print(f"❌ Error in transcription: {e}")
            return "❌ Failed to transcribe the audio. Please try again.", None

    elif input_type == "Image":
        if not uploaded_image:
            return "❌ No image uploaded.", None
        food_description = image_caption(uploaded_image)
        if not food_description or "error" in food_description.lower():
            return "❌ Could not analyze the image properly.", None
        user_question = f"Is {food_description} good to eat?"
        transcription_display = food_description

    if health_condition and health_condition.strip() != "":
        keywords = [
            # English variations
            "eat", "can i eat", "should i eat", "avoid", "safe to eat",
            "food", "foods", "meal", "diet", "drink", "nutrition",
            "what should i eat", "what not to eat", "good for", "bad for",
            "breakfast", "lunch", "dinner", "snack", "healthy", "unhealthy",

            # Arabic variations
            "أكل", "تناول", "هل أستطيع", "هل يمكنني", "هل يجوز", "ينفع أكل",
            "ما الأطعمة", "ما الأكل", "ما الطعام", "تجنب", "ضار", "مفيد",
            "غداء", "فطور", "عشاء", "وجبة", "شرب", "مشروب"
        ]
    
    question_lower = user_question.lower()
    if any(kw in question_lower for kw in keywords):
        user_question += f" I have {health_condition}."


    user_question_en = user_question
    if language_choice == "Arabic":
        user_question_en = translate(user_question, source_lang="ar", target_lang="en")

    try:
        current_history = memory.load_memory_variables({})["chat_history"]
    except Exception as e:
        print(f"⚠️ Failed to load chat history: {e}")
        current_history = []

    try:
        response_en = agent_executor.invoke({"input": user_question_en})['output']
    except Exception as e:
        print(f"❌ Agent error: {e}")
        return "❌ Sorry, something went wrong while processing your question.", transcription_display

    irrelevant_keywords = [
        "cloud", "server", "AI model", "database", "networking", "API",
        "programming", "code", "machine learning", "software"
    ]
    if any(keyword in response_en.lower() for keyword in irrelevant_keywords):
        response_en = "❌ I'm sorry, that doesn't seem related to food or health. Please ask a nutrition-related question."

    response = response_en
    if language_choice == "Arabic" and user_question != user_question_en:
        response = translate(response_en, source_lang="en", target_lang="ar")

    return response, transcription_display

###  Gradio User Interface

In [ ]:
gr_theme_css = """
<style>
@font-face {
    font-family: 'Dubai';
    src: url('Dubai-Medium.ttf') format('truetype');
    font-weight: normal;
    font-style: normal;
}

body, .gradio-container {
    background-color: #ffffff !important;
    font-family: 'Dubai', -apple-system, BlinkMacSystemFont, sans-serif !important;
}

.gradio-container {
    width: 100% !important;
    max-width: 1200px !important;
    margin: 0 auto !important;
    padding: 20px !important;
}

.chat-container {
    background-color: white;
    border-radius: 8px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
    height: calc(100vh - 200px) !important;
    display: flex;
    flex-direction: column;
    overflow: hidden;
}

#chatbot {
    flex-grow: 1;
    overflow-y: auto;
    padding: 20px;
    background-color: #ffffff;
    border-radius: 8px 8px 0 0;
}

/* User messages: background now light green, border green */
.user-message {
    background-color: #e0f7e9 !important;
    border: 2px solid #7dd2a9 !important;
    margin-left: auto;
    margin-right: 10px;
    border-bottom-right-radius: 0;
    color: #333;
    padding: 12px 16px;
    margin-bottom: 16px;
    border-radius: 8px;
    line-height: 1.3;
    max-width: 80%;
    width: fit-content;
    font-size: 14px !important;
}

.bot-message {
    background-color: #d5e7dd !important;
    border: 2px solid #2e8b57 !important;
    margin-right: auto;
    margin-left: 10px;
    border-bottom-left-radius: 0;
    box-shadow: 0 1px 2px rgba(0,0,0,0.1);
    color: #333;
    padding: 12px 16px;
    margin-bottom: 16px;
    border-radius: 8px;
    line-height: 1.3;
    max-width: 80%;
    width: fit-content;
    font-size: 14px !important;
}

.input-area {
    padding: 16px;
    background-color: white;
    border-top: 1px solid #e5e5e5;
    border-radius: 0 0 8px 8px;
    display: flex;
    gap: 10px;
}

#text-input {
    flex-grow: 1;
    border: 1px solid #e5e5e5;
    border-radius: 20px;
    padding: 12px 16px;
    font-size: 16px;
    outline: none;
}

#text-input:focus {
    border-color: #7dd2a9;
    box-shadow: 0 0 0 1px #7dd2a9;
}

button {
    background-color: #7dd2a9 !important;
    color: white !important;
    border: none !important;
    border-radius: 20px !important;
    padding: 12px 20px !important;
    font-size: 16px !important;
    font-weight: 500 !important;
    cursor: pointer !important;
    transition: all 0.2s !important;
}

button:hover {
    background-color: #6bc198 !important;
}

button:disabled {
    background-color: #e5e5e5 !important;
    color: #999 !important;
}

.options-area {
    padding: 12px 16px;
    background-color: white;
    border-bottom: 1px solid #e5e5e5;
    display: flex;
    gap: 12px;
    flex-wrap: wrap;
}

.dropdown, .textbox, .audio, .image {
    margin-bottom: 0 !important;
}

@media (max-width: 768px) {
    .gradio-container {
        padding: 10px !important;
    }
    
    .user-message, .bot-message {
        max-width: 90%;
        font-size: 18px;
        padding: 10px 14px;
    }
    
    .chat-container {
        height: 65vh !important;
    }
    
    #text-input, button {
        font-size: 16px !important;
        padding: 10px 14px !important;
    }
}
</style>
"""

In [ ]:
with gr.Blocks(title="NutriBot: Smart Nutrition Assistant", 
               css=gr_theme_css) as demo:

    with open("nutribotLogo.png", "rb") as f:
        b64 = base64.b64encode(f.read()).decode()
    logo_data_url = f"data:image/png;base64,{b64}"

    gr.Markdown(f"""
    <div style="
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        margin-bottom: 12px;
    ">
        <img 
            src="{logo_data_url}" 
            alt="NutriBot Logo" 
            style="max-width:200px; display:block;"
        />
        <p style="margin: 8px 0 0; text-align: center; color: #666; font-size: 20px;">
            Type your question, speak, or upload food image to get healthy dietary suggestions
        </p>
    </div>
    """
    )

    with gr.Row(elem_classes=["options-area"]):
        language_dropdown = gr.Dropdown(
            choices=["English", "Arabic"], 
            value="English", 
            label="Language",
            interactive=True
        )
        health_input = gr.Textbox(
            label="Health condition (optional)", 
            placeholder="e.g., diabetes, high blood pressure...",
            interactive=True,
            elem_id="health-input"  
        )

    chatbot = gr.Chatbot(
        elem_id="chatbot",
        bubble_full_width=False,
        height=500,
        show_label=False
    )

    with gr.Row(elem_classes=["input-area"]):
        msg = gr.Textbox(
            show_label=False,
            placeholder="Type your nutrition question here...",
            container=False,
            scale=7,
            elem_id="text-input"
        )
        send_btn = gr.Button("Send", variant="primary", scale=1)

    with gr.Accordion("Attach file (image or voice)", open=False):
        with gr.Row():
            with gr.Column():
                gr.HTML("<div style='text-align:center;'>Upload Image</div>")
                image_input = gr.Image(
                    type="filepath", 
                    interactive=True,
                    show_label=False
                )
            with gr.Column():
                gr.HTML("<div style='text-align:center;'>Record or Upload Voice</div>")
                audio_input = gr.Audio(
                    type="filepath", 
                    interactive=True,
                    show_label=False
                )

    status_message = gr.State("")

    def show_processing(chat_history, audio, lang_choice):
        if audio is not None and lang_choice == "Arabic":
            processing_msg = "<div class='arabic-text'>جاري معالجة الصوت العربي... قد يستغرق ذلك وقت أطول من المعتاد.</div>"
        elif audio is not None:
            processing_msg = "Processing audio... Please wait."
        else:
            return chat_history, ""
            
        chat_history.append((None, processing_msg))
        return chat_history, ""

    def respond(message, chat_history, image, audio, health, lang_choice):
        try:
            if image is not None:
                input_type = "Image"
            elif audio is not None:
                input_type = "Voice"
            else:
                input_type = "Text"

            response, transcription_display = chatbot_logic(message, input_type, image, audio, health, lang_choice)

            user_message = message if input_type == "Text" else None
            if transcription_display:
                if input_type == "Image":
                    user_message = f"📷 Image uploaded"  # تغيير النص ليشبه كود 1
                elif input_type == "Voice":
                    user_message = f"🎤 {transcription_display}"

            if lang_choice == "Arabic":
                if user_message:
                    user_message = f"<div class='arabic-text'>{user_message}</div>"
                response = f"<div class='arabic-text'>{response}</div>"

            chat_history.append((user_message, response))
            return "", chat_history, None, None, ""
        except Exception as e:
            print(f"❌ Unexpected error in respond(): {e}")
            error_message = "❌ An error occurred while processing your question. Please try again."
            if lang_choice == "Arabic":
                error_message = "<div class='arabic-text'>❌ حدث خطأ أثناء معالجة سؤالك. حاول مرة أخرى من فضلك.</div>"
            chat_history.append((None, error_message))
            return "", chat_history, None, None, ""

    msg.submit(
        fn=show_processing,
        inputs=[chatbot, audio_input, language_dropdown],
        outputs=[chatbot, status_message]
    ).then(
        fn=respond,
        inputs=[msg, chatbot, image_input, audio_input, health_input, language_dropdown],
        outputs=[msg, chatbot, image_input, audio_input, status_message]
    )

    send_btn.click(
        fn=show_processing,
        inputs=[chatbot, audio_input, language_dropdown],
        outputs=[chatbot, status_message]
    ).then(
        fn=respond,
        inputs=[msg, chatbot, image_input, audio_input, health_input, language_dropdown],
        outputs=[msg, chatbot, image_input, audio_input, status_message]
    )

if __name__ == "__main__":
    demo.launch()